
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

#Setup

### Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Install required libraries
Be sure to click on the **restart** button after running this

In [2]:
!pip install -U --pre tensorflow=="1.*"
!pip install tf_slim
!apt-get update && apt-get upgrade -y
!pip install numpy==1.16 #This version is required due to a bug. After installing this version, the runtime has to be restarted.

     |████████████████████████████████| 110.5MB 69kB/s 
     |████████████████████████████████| 512kB 44.2MB/s 
     |████████████████████████████████| 3.8MB 49.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7ec12b0abb6c8a2210aef93a558c54289922b4fc44a97170d344572f43e49f64
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: 

Make sure you have `pycocotools` installed

In [1]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [2]:
!apt-get install python-pil python-lxml 

!pip install Cython
!pip install contextlib2
#!pip install jupyter
!pip install matplotlib
!pip install wget

import os
import pathlib
import wget


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

os.environ['PYTHONPATH'] += ":/content/models/research/:/content/models/research/slim"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 17 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic

Compile protobufs and install the object_detection package

In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [4]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1113489 sha256=927f25c128bf65a6b81ae71ae49168d56f47d078013ead4c26830a7b11ea9397
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4rufpn3/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


# Imports & Configuration

In [5]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import *
wget.download(PRETRAINED_MODEL_CONFIG,CONFIG_DIR)

--2020-06-16 19:18:38--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6151 (6.0K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   6.01K  --.-KB/s    in 0s      

2020-06-16 19:18:38 (55.6 MB/s) - ‘constants.py’ saved [6151/6151]



'/content/Face-Mask-Detection/training/config//pipeline (1).config'

## Copy dataset and extract

In [7]:
!gsutil cp {DRIVE_DEV}model_checkpointsv1.zip /content/
!unzip -j model_checkpointsv1.zip -d {CKPT_DIR}

Archive:  model_checkpointsv1.zip
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-11244.meta  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-7477.data-00000-of-00001  
  inflating: /content/Face-Mask-Detection/training/checkpoints/events.out.tfevents.1592316220.8ef3e07ceeae  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-12496.index  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-9988.index  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-9988.meta  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-12496.meta  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-11244.data-00000-of-00001  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-9988.data-00000-of-00001  
  inflating: /content/Face-Mask-Detection/training/checkpoints/model.ckpt-7477.index  
  inflating: /content/Face-Mask

# Conversions

## Export inference graph

Use this option if you want to convert to TFjs later

In [10]:
!python /content/models/research/object_detection/export_inference_graph.py \
  --pipeline_config_path={CONFIG_DIR}pipeline.config \
  --trained_checkpoint_prefix={CKPT_DIR}/model.ckpt-12496 \
  --output_directory={INF_GRAPH} \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

Instructions for updating:
Please use `layer.__call__` method instead.
W0616 19:23:37.955227 140236934023040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:23:40.478758 140236934023040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:23:40.518374 140236934023040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:23:40.557896 140236934023040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

## Export TF Lite SSD inference graph

Use this option for TF Lite conversion

In [11]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="{CONFIG_DIR}/pipeline.config" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-12496" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0616 19:25:19.905396 140174986577792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:25:22.344484 140174986577792 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:25:22.488449 140174986577792 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 19:25:22.520295 140174986577792 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

## Convert graph to TF Lite

In [12]:
!tflite_convert \
--output_file="{TF_LITE_MODEL}/detect.tflite" \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--inference_type=QUANTIZED_UINT8 \
--input_arrays="{INPUT_TENSORS}" \
--output_arrays="{OUTPUT_TENSORS}"  \
--mean_values=128 \
--std_dev_values=128 \
--input_shapes=1,320,320,3 \
--change_concat_input_ranges=false \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops \
--output_format=TFLITE \
--default_ranges_min=0 \
--default_ranges_max=255

2020-06-16 19:27:24.673070: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-16 19:27:24.677130: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-06-16 19:27:24.677333: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2388a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-16 19:27:24.677365: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


# Save 

## Archive the inference graph

In [13]:
!zip -r part5-inf-graphv1.zip {INF_GRAPH}
!gsutil cp part5-inf-graphv1.zip {DRIVE_DEV}

  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/ (stored 0%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/pipeline.config (deflated 69%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/model.ckpt.index (deflated 67%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/model.ckpt.meta (deflated 94%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/model.ckpt.data-00000-of-00001 (deflated 6%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/checkpoint (deflated 42%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/saved_model/ (stored 0%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/saved_model/saved_model.pb (deflated 12%)
  adding: content/Face-Mask-Detection/training/trained_graphs/inference_graph/saved_model/variables/ (stored 0%)
  adding: content/

## Archive the TF Lite graph

In [14]:
!zip -r part5-tflite-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp part5-tflite-graphv1.zip {DRIVE_DEV}

  adding: content/Face-Mask-Detection/training/trained_graphs/tf_lite_graph/ (stored 0%)
  adding: content/Face-Mask-Detection/training/trained_graphs/tf_lite_graph/tflite_graph.pb (deflated 9%)
  adding: content/Face-Mask-Detection/training/trained_graphs/tf_lite_graph/tflite_graph.pbtxt (deflated 56%)
Copying file://part5-tflite-graphv1.zip...
/ [1 files][ 28.1 MiB/ 28.1 MiB]                                                
Operation completed over 1 objects/28.1 MiB.                                     


## Archive the TF Lite model

In [15]:
!zip -r part5-tflite-modelv1.zip {TF_LITE_MODEL}
!gsutil cp part5-tflite-modelv1.zip {DRIVE_DEV}

  adding: content/Face-Mask-Detection/training/trained-models/tflite/ (stored 0%)
  adding: content/Face-Mask-Detection/training/trained-models/tflite/detect.tflite (deflated 31%)
Copying file://part5-tflite-modelv1.zip...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.                                      
